In [ ]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report


with open('data.json', 'r') as f:
    data = json.load(f)

intents = data['intents']

X = []
y = []

for intent in intents:
    for pattern in intent['patterns']:
        X.append(pattern)
        y.append(intent['tag'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#tfidf 
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


labels = list(set(y_train))
label_to_idx = {label: idx for idx, label in enumerate(labels)}
y_train_numeric = np.array([label_to_idx[label] for label in y_train])
y_test_numeric = np.array([label_to_idx[label] for label in y_test])

#hyperparameters for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'] + [0.001, 0.01, 0.1, 1, 10]
}

#model
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train_numeric)

#best model from GridSearchCV
best_svm_model = grid_search.best_estimator_
y_pred_numeric = best_svm_model.predict(X_test_tfidf)
pred_labels = [labels[idx] for idx in y_pred_numeric]

print(classification_report(y_test, pred_labels))



In [7]:
import random
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


with open('data.json', 'r') as json_data:
    intents = json.load(json_data)

#extract data from json
X = []
y = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        X.append(pattern)
        y.append(intent['tag'])

# vectorize data
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# labels to numerical values
labels = list(set(y))
label_to_idx = {label: idx for idx, label in enumerate(labels)}
y_numeric = np.array([label_to_idx[label] for label in y])

# hyperparameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'] + [0.001, 0.01, 0.1, 1, 10]
}

#SVM Model 
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_tfidf, y_numeric)

# get best model from GridSearchCV
best_svm_model = grid_search.best_estimator_

#classify intent the user input 
def classify_intent(user_input):
    user_input_tfidf = vectorizer.transform([user_input])
    predicted_label_numeric = best_svm_model.predict(user_input_tfidf)
    predicted_label = labels[predicted_label_numeric[0]]
    return predicted_label



c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [6]:

def chatbot_loop():
    bot_name = "Chatbot"
    print("Let's chat! (type 'Quit' to exit)")
    while True:
        user_input = input("You: ")
        if user_input == "Quit":
            break
        
        user_input_tfidf = vectorizer.transform([user_input]) 
        intent = classify_intent(user_input)
        
        similarity_scores = user_input_tfidf.dot(X_tfidf.T).toarray()[0]
        max_similarity = np.max(similarity_scores)

        print(f"You: {user_input}")
        if max_similarity < 0.5:
            print(f"{bot_name}: I do not understand...")
        else:
            for intent_data in intents['intents']:
                if intent_data['tag'] == intent:
                    responses = intent_data['responses']
                    response = f"{bot_name}: " + random.choice(responses)
                    break
            else:
                response = f"{bot_name}: I do not understand..."

            print(response)

chatbot_loop()


Let's chat! (type 'Quit' to exit)
You: Hi
Chatbot: Hi, welcome to Predoctoral Implant Program. How may I assist you today?
You: What instruments for IOD?
Chatbot: You need to get a fixed cassette from the window, xray sensor and implant cassette from the window. You need to know which implant system that you the patient has. Please look the patient   s information from the axium.
You: IOD Instruments?
Chatbot: You need to get a fixed cassette from the window, xray sensor and implant cassette from the window. You need to know which implant system that you the patient has. Please look the patient   s information from the axium.
You: STI Scheduling
Chatbot: I do not understand...
You: STI Schedule
Chatbot: I do not understand...
You: How to have a schedule for STI
Chatbot: I do not understand...
You: How long for follow up treatment 
Chatbot: I do not understand...
You: How long for follow up IOD
Chatbot: I do not understand...
You: Who needs to approve codes
Chatbot: Your restorative fac